In [2]:
import pandas as pd
import numpy as np

# Clean MovieLens 100K Dataset
#### Ratings

In [14]:
# read data and add column headers
rating_df = pd.read_csv('data/raw/ml-100k/u.data', sep='\t', header=None)
rating_df.columns = [x.strip() for x in 'user id | item id | rating | timestamp'.split('|')]

# keep only top 100 most popular movies
rating_df = rating_df[
    rating_df['item id'].isin(rating_df['item id'].value_counts().head(100).index)
]

rating_df.rename(columns={'user id': 'user_id', 'item id': 'item_id'}, inplace=True)

print('Average raw rating:', rating_df['rating'].mean())
print('Total ratings:', len(rating_df))

rating_df.to_csv('./data/clean/movie-lens-100k-ratings.csv', index=False)
rating_df.head()

Average raw rating: 3.828104640673549
Total ratings: 29931


,user_id,item_id,rating,timestamp
1,186,302,3,891717742
6,115,265,2,881171488
10,62,257,2,879372434
12,200,222,5,876042340
22,299,144,4,877881320


#### Users

In [12]:
user_df = pd.read_csv('data/raw/ml-100k/u.user', sep='|', header=None)
user_df.columns = [x.strip() for x in 'user id | age | gender | occupation | zip code'.split('|')]

user_df['decades_lived'] = (10 * np.floor((user_df.age / 10))).astype(int).astype(str)
user_df.drop(columns=['age', 'zip code'], inplace=True)

user_df.rename(columns={'user id': 'user_id'}, inplace=True)

print(user_df.nunique())
user_df.to_csv('./data/clean/movie-lens-100k-users.csv', index=False)
user_df.head()

user_id          943
gender             2
occupation        21
decades_lived      8
dtype: int64


,user_id,gender,occupation,decades_lived
0,1,M,technician,20
1,2,F,other,50
2,3,M,writer,20
3,4,M,technician,20
4,5,F,other,30


In [10]:
item_df = pd.read_csv('data/raw/ml-100k/u.item', sep='|', header=None, encoding = "ISO-8859-1")

cols = """
    movie id | movie title | release date | video release date |
    IMDb URL | unknown | Action | Adventure | Animation |
    Children's | Comedy | Crime | Documentary | Drama | Fantasy |
    Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
    Thriller | War | Western 
"""
# update columns names of dataframe
item_df.columns = [x.strip() for x in cols.split('|')]

# replace date with decade of release
item_df['release_decade'] = [str(int(x)) if ~np.isnan(x) else np.nan for x in 10 * np.floor(pd.to_datetime(item_df['release date']).dt.year / 10)]

# drop date
item_df.drop(columns=['release date', 'video release date', 'IMDb URL', 'movie title'], inplace=True)

# reset dummies
item_df.set_index(['movie id'],inplace=True)
item_df = item_df[item_df==1].stack().reset_index().drop(0, axis=1)\
    .rename(columns={'level_1': 'genre'})\
    .merge(item_df['release_decade'], left_on='movie id', right_index=True, how='left')\
    .rename(columns={'movie id': 'item_id'})

item_df['release_decade'] = item_df['release_decade'].astype(str)

print(item_df.nunique())
item_df.to_csv('./data/clean/movie-lens-100k-items.csv', index=False)
item_df.head()

item_id           1682
genre               19
release_decade       9
dtype: int64


,item_id,genre,release_decade
0,1,Animation,1990
1,1,Children's,1990
2,1,Comedy,1990
3,2,Action,1990
4,2,Adventure,1990
